In [1]:
import numpy as np
from sklearn import datasets

In [18]:
digits = datasets.load_digits()
X = digits.data
y = digits.target

(1797, 64)

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=666)

In [8]:
from sklearn.neighbors import KNeighborsClassifier

best_score, best_p, best_k = 0, 0, 0
for k in range(2, 11):
    for p in range(1, 6):
        knn_clf = KNeighborsClassifier(weights="distance", n_neighbors=k, p=p)
        knn_clf.fit(X_train, y_train)
        score = knn_clf.score(X_test, y_test)
        if score > best_score:
            best_score, best_p, best_k = score, p, k
            
print("best K = ", best_k)
print("best P = ", best_p)
print("best Score = ", best_score)

best K =  3
best P =  2
best Score =  0.9860917941585535


In [9]:
# 直接调用包就行，可以自动为我们完成交叉验证的过程
from sklearn.model_selection import cross_val_score

knn_clf = KNeighborsClassifier()
cross_val_score(knn_clf, X_train, y_train)

array([0.99537037, 0.98148148, 0.97685185, 0.97674419, 0.97209302])

In [12]:
best_score, best_p, best_k = 0, 0, 0
for k in range(2, 11):
    for p in range(1, 6):
        knn_clf = KNeighborsClassifier(weights="distance", n_neighbors=k, p=p)
        # 只需在这里变一下即可
        # 注意，此时测试数据是不参与的
        scores = cross_val_score(knn_clf, X_train, y_train)
        score = np.mean(scores)
        if score > best_score:
            best_score, best_p, best_k = score, p, k

# 交叉验证得到的超参数更可靠
print("best K = ", best_k)
print("best P = ", best_p)
print("best Score = ", best_score)

best K =  2
best P =  2
best Score =  0.9851507321274763


In [14]:
# 我们使用交叉验证只是想获得最好的 p,k 超参数而已
# 然后我们就用这些超参数定义自己的模型
# 也就是 k=2, p=2
best_knn_clf = KNeighborsClassifier(weights="distance", n_neighbors=2, p=2)

In [15]:
# 我们只在训练完模型后，再用模型没接触过的测试数据集查看准确度
best_knn_clf.fit(X_train, y_train)
best_knn_clf.score(X_test, y_test)

0.980528511821975

In [17]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {
        'weights': ['distance'],
        'n_neighbors': [i for i in range(2, 11)],
        'p': [i for i in range(1, 6)]
    }
]

# GridSearchCV 会自动交叉验证
# 分成5份，而参数组合有 9*5=45 种，共225次训练
grid_search = GridSearchCV(knn_clf, param_grid, verbose=1)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 45 candidates, totalling 225 fits


GridSearchCV(estimator=KNeighborsClassifier(n_neighbors=10, p=5,
                                            weights='distance'),
             param_grid=[{'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9, 10],
                          'p': [1, 2, 3, 4, 5], 'weights': ['distance']}],
             verbose=1)

In [22]:
print(grid_search.best_score_)
print(grid_search.best_params_)

0.9851507321274763
{'n_neighbors': 2, 'p': 2, 'weights': 'distance'}


In [23]:
# 所以可以直接带入网格搜索得出的最佳超参数训练模型
best_knn_clf = grid_search.best_estimator_
best_knn_clf.score(X_test, y_test)

0.980528511821975